In [3]:
from keybert import KeyBERT

keybert_ext = KeyBERT()

In [2]:
from proj import dataloader

dataset = dataloader.load_billsum(
    "local_datasets"
)

In [ ]:
dataset_train = dataset["train"]
dataset_test = dataset["test"]

In [7]:
train_text_kw = keybert_ext.extract_keywords([*map(lambda s: s.lower(), dataset_train["text"])])
train_sum_kw = keybert_ext.extract_keywords([*map(lambda s: s.lower(), dataset_train["summary"])])

test_text_kw = keybert_ext.extract_keywords([*map(lambda s: s.lower(), dataset_test["text"])])
test_sum_kw = keybert_ext.extract_keywords([*map(lambda s: s.lower(), dataset_test["summary"])])

In [17]:
import pandas as ps

def csv_format_script(kw_lists):
    output_list = []
    for kw_list in kw_lists:
        probs = []
        words = []
        for kw_pair in kw_list:
            probs.append(kw_pair[1])
            words.append(kw_pair[0])
        
        words.extend(probs)
        output_list.append(words)
    return output_list
    

def add_titles(kw_lists, titles_list):
    output_list = []
    
    for kw_list, title in zip(kw_lists, titles_list):
        kw_list.insert(0, title)
        output_list.append(kw_list)
        
    return output_list


def converter_to_ps_script(kw_lists, titles_list):
    max_length = len(kw_lists[0])
    columns = ["title"]
    columns.extend(f"keyword{i}" for i in range(max_length))
    columns.extend(f"kw_prob{i}" for i in range(max_length))
    return ps.DataFrame(add_titles(csv_format_script(kw_lists), titles_list), columns=columns)
    

train_text_kw_ps = converter_to_ps_script(train_text_kw, dataset_train["title"])
train_sum_kw_ps = converter_to_ps_script(train_sum_kw, dataset_train["title"])

test_text_kw_ps = converter_to_ps_script(test_text_kw, dataset_test["title"])
test_sum_kw_ps = converter_to_ps_script(test_sum_kw, dataset_test["title"])

In [18]:
# Collect the data
import os
import csv

os.mkdir(".temp/")
os.mkdir(".temp/keywords/")

os.mkdir(".temp/keywords/billsum/")

train_sum_kw_ps.to_csv(".temp/keywords/billsum/train_sum_kw.csv", index=False)
train_text_kw_ps.to_csv(".temp/keywords/billsum/train_text_kw.csv", index=False)
test_text_kw_ps.to_csv(".temp/keywords/billsum/test_text_kw.csv", index=False)
test_sum_kw_ps.to_csv(".temp/keywords/billsum/test_sum_kw.csv", index=False)